In [10]:
# Import dependencies

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [11]:
# Read in the weather data / cities CSV that we created in Part 1

cities = pd.read_csv('Output_CSV/weather_data.csv', encoding="utf-8")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kapaa,40,US,1592656320,83,22.08,-159.32,77.00,12.75
1,1,Portland,100,US,1592656038,77,45.52,-122.68,64.99,3.29
2,2,Saldanha,0,ZA,1592656322,68,-33.01,17.94,66.20,8.05
3,3,Mar del Plata,61,AR,1592656130,70,-38.00,-57.56,52.00,4.70
4,4,Nānākuli,40,US,1592656324,73,21.39,-158.15,77.00,3.36


In [3]:
# Create humidity heatmap

humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Create a new dataframe that fits weather criteria
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,19,Arraial do Cabo,0,BR,1592656342,83,-22.97,-42.03,73.40,6.93
1,39,Seara,0,PT,1592656366,59,41.18,-7.80,75.00,1.01
2,49,Krasnoarmeysk,0,RU,1592656377,47,51.02,45.70,79.86,7.70
3,60,Jardim,0,BR,1592656390,57,-21.48,-56.14,75.16,7.16
4,78,Port Alfred,0,ZA,1592656411,67,-33.59,26.89,71.01,1.99


In [6]:
# Store everything into variable named hotel_df, set parameters, and begin to plot markers on top of heatmap

hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,19,Arraial do Cabo,0,BR,1592656342,83,-22.97,-42.03,73.40,6.93,Cabo Frio
1,39,Seara,0,PT,1592656366,59,41.18,-7.80,75.00,1.01,Peso da Régua
2,49,Krasnoarmeysk,0,RU,1592656377,47,51.02,45.70,79.86,7.70,Krasnoarmeysk
3,60,Jardim,0,BR,1592656390,57,-21.48,-56.14,75.16,7.16,Jardim
4,78,Port Alfred,0,ZA,1592656411,67,-33.59,26.89,71.01,1.99,Port Alfred


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [8]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))